In [ ]:
\list

## Exploring 'keywords_all' Column
A brief exploration of the 'keywords_all' column in the 'data_nerd_jobs' table, focusing on the structure and content of the data.

In [ ]:
SELECT keywords_all FROM public_job_listings.data_nerd_jobs;

## Unpacking JSON Objects in 'keywords_all'
Extracting job skills from the JSON objects in the 'keywords_all' column of the 'data_nerd_jobs' table.

In [ ]:
SELECT
  keyword.element
FROM
  public_job_listings.data_nerd_jobs,
  UNNEST(keywords_all.list) AS keyword
LIMIT 100;

## Median Salary for Top 10 Skills
Calculating the median salary for the top 10 skills listed in the 'keywords_all' column, using the 'salary_year' column from the 'data_nerd_jobs' table.

## Median Salary for Top 10 Skills
Calculating the median salary for the top 10 skills listed in the 'keywords_all' column using the 'salary_year' column.

In [ ]:
WITH TopSkills AS (
  SELECT
    keyword.element AS skill,
    COUNT(*) AS count
  FROM
    public_job_listings.data_nerd_jobs,
    UNNEST(keywords_all.list) AS keyword
  GROUP BY skill
  ORDER BY COUNT(*) DESC
  LIMIT 10
),
MedianSalaries AS (
  SELECT
    ts.skill,
    PERCENTILE_CONT(salary_year, 0.5) OVER (PARTITION BY ts.skill) AS median_salary,
    MAX(ts.count) OVER (PARTITION BY ts.skill) AS skill_count
  FROM
    public_job_listings.data_nerd_jobs,
    UNNEST(keywords_all.list) AS keyword
  JOIN TopSkills ts ON keyword.element = ts.skill
)
SELECT
  skill,
  median_salary,
  skill_count
FROM
  MedianSalaries
GROUP BY skill, median_salary, skill_count;

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Setting the plot style
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Preparing data
data = sql_df_ghfh[['skill', 'median_salary', 'skill_count']].sort_values('median_salary', ascending=False)

# Creating the bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x='median_salary', y='skill', data=data, palette="Blues_r")

# Adding the count inside the bars
for index, value in enumerate(data['median_salary']):
    plt.text(value, index, str(data['skill_count'].iloc[index]), color='white', va='center')

plt.title('Median Salary and Job Postings for Top 10 Skills')
plt.xlabel('Median Salary ($)')
plt.ylabel('Skill')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Setting the plot style
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Preparing data
data = sql_df_ghfh[['skill', 'median_salary', 'skill_count']].sort_values('median_salary', ascending=False)

# Creating the bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x='median_salary', y='skill', data=data, palette="Blues_r")

# Adding the count inside the bars
for index, value in enumerate(data['median_salary']):
    plt.text(value, index, f'{data["skill_count"].iloc[index]} postings', color='white', va='center')

plt.title('Median Salary for Top 10 Skills with Job Postings Count')
plt.xlabel('Median Salary ($)')
plt.ylabel('Skill')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Setting the plot style
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Preparing data
data = sql_df_ghfh[['skill', 'median_salary', 'skill_count']].sort_values('median_salary', ascending=False)

# Creating the bar plot
plt.figure(figsize=(14, 10))
sns.barplot(x='median_salary', y='skill', data=data, palette="Blues_r")

# Adding the count inside the bars
for index, value in enumerate(data['median_salary']):
    plt.text(value - 5000, index, f'{data["skill_count"].iloc[index]} postings', color='white', va='center')

plt.title('Median Salary for Top 10 Skills with Job Postings Count')
plt.xlabel('Median Salary ($)')
plt.ylabel('Skill')
plt.show()

## Median Salary for All Skills
Calculating the median salary for all skills listed in the 'keywords_all' column using the 'salary_year' column.

In [ ]:
SELECT
  skill,
  COUNT(*) AS skill_count,
  ROUND(AVG(salary_year)) AS median_salary
FROM (
  SELECT
    keyword.element AS skill,
    salary_year
  FROM
    public_job_listings.data_nerd_jobs,
    UNNEST(keywords_all.list) AS keyword
)
GROUP BY skill;

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

# Preparing data
plot_data = sql_df_xcmj.dropna()

# Creating the scatter plot
fig = px.scatter(plot_data, x='median_salary', y='skill_count', hover_data=['skill'],
                 title='Scatter Plot of Salary vs Count', labels={'median_salary': 'Median Salary', 'skill_count': 'Skill Count'})

fig.show()

In [ ]:
SELECT
  job_title_final,
  keyword.element AS skill,
  COUNT(*) AS skill_count,
  ROUND(AVG(salary_year)) AS median_salary
FROM
  public_job_listings.data_nerd_jobs,
  UNNEST(keywords_all.list) AS keyword
GROUP BY job_title_final, skill;

In [ ]:
SELECT
  keyword.element AS skill,
  job_title_final,
  COUNT(*) AS skill_count,
  ROUND(AVG(salary_year)) AS median_salary
FROM
  public_job_listings.data_nerd_jobs,
  UNNEST(keywords_all.list) AS keyword
GROUP BY skill, job_title_final;

In [ ]:
SELECT
  keyword.element AS skill,
  job_title_final,
  COUNT(*) AS skill_count,
  ROUND(AVG(salary_year)) AS median_salary
FROM
  public_job_listings.data_nerd_jobs,
  UNNEST(keywords_all.list) AS keyword
GROUP BY skill, job_title_final;

In [ ]:
import plotly.express as px

# Preparing data
plot_data = sql_df_arwr.dropna()

# Creating the scatter plot with a slicer for job_title_final
fig = px.scatter(plot_data, x='median_salary', y='skill_count', color='job_title_final', hover_data=['skill', 'job_title_final'],
                 title='Scatter Plot of Salary vs Skill Count with Job Title Filter', labels={'median_salary': 'Median Salary', 'skill_count': 'Skill Count'})

fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply MinMax normalization within each job title category
sql_df_arwr['normalized_multiplier'] = sql_df_arwr.groupby('job_title_final')['skill_multiplier'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())

sql_df_arwr.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter data for 'Data Analyst' job title
analyst_data = sql_df_arwr[sql_df_arwr['job_title_final'] == 'Data Analyst']

# Sort and select top 10 skills based on normalized multiplier
top_skills = analyst_data.sort_values('normalized_multiplier', ascending=False).head(10)

# Setting the plot style
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Creating the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='normalized_multiplier', y='skill', data=top_skills, palette="Blues_r")

plt.title('Top 10 Skills for Data Analysts Based on Normalized Multiplier')
plt.xlabel('Normalized Multiplier')
plt.ylabel('Skill')
plt.show()

In [ ]:
import plotly.express as px

# Creating the interactive bar plot with a slicer for job_title_clean
fig = px.bar(sql_df_arwr, x='normalized_multiplier', y='skill', color='job_title_clean',
             title='Skills Based on Normalized Multiplier with Job Title Filter',
             labels={'normalized_multiplier': 'Normalized Multiplier', 'skill': 'Skill'},
             hover_data=['job_title_clean', 'skill', 'normalized_multiplier'])

fig.show()